In [ ]:
alldata = dict()

In [ ]:
# %matplotlib nbagg
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patch
from matplotlib.patches import Polygon

plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True

import pandas as pd

import numpy as np
import scipy as sp
# import scipy.optimize 
import scipy.optimize as opt
from scipy.optimize import curve_fit
from scipy.misc import face
from scipy import interpolate

import glob
import csv
import re
import sys 
import os
import copy
import time
import h5py
import skimage.feature
import skimage.filters
import skimage.measure
import socket  
import itertools
from tqdm.notebook import tqdm as tqdm

from skimage import filters


from IPython.display import display, clear_output, Markdown
from PIL import Image, ImageOps

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

import cv2

udir = '/cds/home/d/diegotur/UED/'

if udir not in sys.path:
    sys.path.append(udir)

import ued_dt3 as ued_dt
try:
    import pyFAI, pyFAI.detectors
    from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
except:
    print('No pyFAI library found. If you want to do azimuthal integrations - install it!')

import dt_functions as dt
import I_UED_dt


import lmfit
from lmfit import Model, Parameters

import imp

In [ ]:
import dill as pickle

In [ ]:
# Loading binned data with static analysis done. 
with open('debye_waller_data/dif_big_dataset_DW.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

dif = loaded_dict

# Diffuse Scattering Analysis

## Initialisation of the analysis

### BZ size

In [ ]:
s = 0 
len_test = len(dif.fnames)
for i in np.arange(len_test):
    i0_delay = float(dif.fnames[i][106:106+12]) # delay in mm 
    fname_delay = float(dif.fnames_I0[i][95:95+8] ) # delay in mm 
    
    i0_delay = ued_dt.mm_to_ps(i0_delay, zero = dif.t0)
    fname_delay = ued_dt.mm_to_ps(fname_delay, zero = dif.t0)

    s += np.abs(i0_delay - fname_delay)


print('precision is on the order of :')
print(str(s/len_test) + ' ps')
    

In [ ]:
bragg = [1,0,0]
sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
fit1 = dif.bragg_fits_binned_delays[:,sel_bragg,:]

bragg2 = [2,0,0]
sel_bragg2 = (dif.allBragg_indices_2[:,0] == bragg2[0])&(dif.allBragg_indices_2[:,1] == bragg2[1])
fit2 = dif.bragg_fits_binned_delays[:,sel_bragg2,:]

In [ ]:
dif.bz_len = np.sqrt(np.sum((fit1[0,0,1:3] - fit2[0,0,1:3])**2))

In [ ]:
dif.sel_bragg_in_im2 = (
    (dif.allBragg_coord_2[:,0] > 0  )&
    (dif.allBragg_coord_2[:,0] < dif.first_image.shape[0]  )&
    (dif.allBragg_coord_2[:,1] > 0  )&
    (dif.allBragg_coord_2[:,1] < dif.first_image.shape[1]  )
)

### Get the coordinates of the different M and X points 

In [ ]:
M_coords = np.zeros((len(dif.allBragg_indices_2),2))
M2_coords = np.zeros((len(dif.allBragg_indices_2),2))
M3_coords = np.zeros((2,2))
X1_coords = np.zeros((len(dif.allBragg_indices_2),2))
X2_coords = np.zeros((len(dif.allBragg_indices_2),2))
X3_coords = np.zeros((len(dif.allBragg_indices_2),2))
X4_coords = np.zeros((len(dif.allBragg_indices_2),2))

M_int = np.zeros(len(dif.allBragg_indices_2))
M2_int = np.zeros(len(dif.allBragg_indices_2))
X1_int = np.zeros(len(dif.allBragg_indices_2))
X2_int = np.zeros(len(dif.allBragg_indices_2))


for n, hkl in enumerate(dif.allBragg_indices_2):

    if dif.sel_bragg_in_im2[n]:
        bragg_coord = dif.allBragg_coord_2[n]
        i= hkl[0]
        j= hkl[1]
        sel_bragg_p1_p0 = (dif.allBragg_indices_2[:,0] == i+1)&(dif.allBragg_indices_2[:,1] == j)&dif.sel_bragg_in_im2
        sel_bragg_p0_p1 = (dif.allBragg_indices_2[:,0] == i)&(dif.allBragg_indices_2[:,1] == j+1)&dif.sel_bragg_in_im2
        sel_bragg_m1_p0 = (dif.allBragg_indices_2[:,0] == i-1)&(dif.allBragg_indices_2[:,1] == j)&dif.sel_bragg_in_im2
        sel_bragg_p0_m1 = (dif.allBragg_indices_2[:,0] == i)&(dif.allBragg_indices_2[:,1] == j-1)&dif.sel_bragg_in_im2        
        
        sel_bragg_p1_p1 = (dif.allBragg_indices_2[:,0] == i+1)&(dif.allBragg_indices_2[:,1] == j+1)&dif.sel_bragg_in_im2

        if sel_bragg_p1_p0.any():
            X1_coords[n] = 0.5*bragg_coord + 0.5*dif.allBragg_coord_2[sel_bragg_p1_p0] 
        if sel_bragg_p0_p1.any():
            X2_coords[n] = 0.5*bragg_coord + 0.5*dif.allBragg_coord_2[sel_bragg_p0_p1]

        if sel_bragg_m1_p0.any():
            X3_coords[n] = 0.5*bragg_coord + 0.5*dif.allBragg_coord_2[sel_bragg_m1_p0] 
        if sel_bragg_p0_m1.any():
            X4_coords[n] = 0.5*bragg_coord + 0.5*dif.allBragg_coord_2[sel_bragg_p0_m1]                
            
        if sel_bragg_p1_p1.any():
            M_coords[n] = 0.5*bragg_coord + 0.5*dif.allBragg_coord_2[sel_bragg_p1_p1]



In [ ]:
x_point_coord_df = pd.DataFrame(data = {'h': dif.allBragg_indices_2[:,0], 'k': dif.allBragg_indices_2[:,1], 'l': dif.allBragg_indices_2[:,2],
                               'x_coord':dif.allBragg_coord_2[:,0], 'y_coord':dif.allBragg_coord_2[:,1], 
                               'X_pt1_x_coord':X1_coords[:,0], 'X_pt1_y_coord':X1_coords[:,1],
                               'X_pt2_x_coord':X2_coords[:,0], 'X_pt2_y_coord':X2_coords[:,1],
                               'X_pt3_x_coord':X3_coords[:,0], 'X_pt3_y_coord':X3_coords[:,1],
                               'X_pt4_x_coord':X4_coords[:,0], 'X_pt4_y_coord':X4_coords[:,1],
                              })


In [ ]:
h, k, l = dif.allBragg_indices_2[0]
print(i)
# load point indices
A,B,C,D,E,F,G,H = I_UED_dt.select_braggs_Bragg(h=h, k=k, allBragg_indices_2=dif.allBragg_indices_2)

In [ ]:
test_array = np.arange(dif.allBragg_indices_2.shape[0])

for i in test_array:
    if i in test_array:
        h, k, l = dif.allBragg_indices_2[i]
        print(i)
        # load point indices
        A,B,C,D,E,F,G,H = I_UED_dt.select_braggs_Bragg(h=h, k=k, allBragg_indices_2=dif.allBragg_indices_2)
        I_UED_dt.apply_symmetry(x_point_coord_df, A,B,C,D,E,F,G,H)

        if (A in test_array):
            test_array = np.setdiff1d(test_array, A)
        if (B in test_array):
            test_array = np.setdiff1d(test_array, B)
        if (C in test_array):
            test_array = np.setdiff1d(test_array, C)
        if (D in test_array):
            test_array = np.setdiff1d(test_array, D)
        if (E in test_array):
            test_array = np.setdiff1d(test_array, E)
        if (F in test_array):
            test_array = np.setdiff1d(test_array, F)
        if (G in test_array):
            test_array = np.setdiff1d(test_array, G)
        if (H in test_array):
            test_array = np.setdiff1d(test_array, H)
            


In [ ]:
h = 2
k = 0
A,B,C,D,E,F,G,H = I_UED_dt.select_braggs_Bragg(h=h, k=k, allBragg_indices_2=dif.allBragg_indices_2)
# A_M,B_M,C_M,D_M,E_M,F_M,G_M,H_M = select_braggs(h=h, k=k, allBragg_indices_2=allBragg_indices_2)

## Look and align diffuse images

In [ ]:
dif.alinged_imgs = np.zeros_like(dif.binned_images)
dif.alingment_warp_matrices = np.zeros( (len(dif.bincenters), *np.eye(3, 3, dtype=np.float32).shape) )

for i, image in enumerate(dif.binned_images):
    
    print(i)

    
    sz = dif.first_image.shape

    # warp_mode = cv2.MOTION_TRANSLATION
    warp_mode = cv2.MOTION_HOMOGRAPHY
    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    
    warp_matrix = np.asarray([[ 9.9841493e-01,  3.0566192e-05,  7.2229356e-01],
       [-9.8849479e-05,  9.9855351e-01,  7.7670795e-01],
       [-1.7143270e-07,  4.6574517e-08,  1.0000000e+00]], dtype=np.float32)
    
    # Specify the number of iterations.
    number_of_iterations = 200;
    
    # Specify the threshold of the increment
    # in the coarrelation coefficient between two iterations
    termination_eps = 1e-5;
    
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)
    
    # Run the ECC algorithm. The results are stored in warp_matrix.
    (cc, warp_matrix) = cv2.findTransformECC (

        dif.binned_images[0].astype(np.float32), 
        dif.binned_images[i].astype(np.float32),

        warp_matrix, warp_mode, criteria, 
        inputMask = None, )
    
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        # Use warpPerspective for Homography
        aligned_image = cv2.warpPerspective (dif.binned_images[i].astype(np.float32), warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else :
        # Use warpAffine for Translation, Euclidean and Affine
        aligned_image = cv2.warpAffine(dif.binned_images[i].astype(np.float32), warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

    
    dif.alinged_imgs[i] = aligned_image
    dif.alingment_warp_matrices[i] = warp_matrix
    
    
    
#     np.save(f'recenter_data/alingned_img_{i}', aligned_image)
#     np.save(f'recenter_data/wrap_matric_{i}', warp_matrix)
    


# np.save(f'recenter_data/All_alingned_imgs', alinged_imgs)
# np.save(f'recenter_data/All_warp_matrices', alingment_warp_matrices)

    

In [ ]:
# Show final results
idx = 24
# idx = 18

fig, ax2 = plt.subplots(ncols= 1, figsize = (2.5,2.5), dpi = 300)
im2 = ax2.imshow(dif.alinged_imgs[idx] - dif.alinged_imgs[0], vmin = -.025, vmax= .025, cmap = 'seismic')
# ax2.set_title(f'Scaled diffuse, {np.round(dif.bincenters[idx], 4)} ps')
ax2.set_xticks([]);
ax2.set_yticks([]);
cb = plt.colorbar(im2, ax=[ax2], location='top')
# cb = plt.colorbar(im2 ,ax = [ax2], location = 'left') 

# cb.set_xlabel('test')
# ax2.xaxis.set_ticks_position('top')

cb.ax.set_xlabel(r'$I-I_0$ (arb units)')

# print({np.round(dif.bincenters[idx], 4)} )

# fig.colorbar(im, orientation="horizontal", pad=0.2)
fig.savefig('colorbar_test.jpeg', dpi=300)

In [ ]:

#create subplot
fig = plt.figure()
ax = fig.add_subplot(111)
axp = ax.imshow(np.random.randint(0, 10,( 10, 10)))
ax.set_title('Colorbar on left')
 
#adding colorbar and its position
cb = plt.colorbar(axp ,ax = [ax], location = 'left')
plt.show()

In [ ]:
time_idx = 0
for i in np.arange(7):
    for j in np.arange(5):
#         im = axes[i,j].imshow(dif.alinged_imgs[time_idx] - dif.alinged_imgs[0:5].mean(axis = 0), vmin = -.025, vmax= .025, cmap = 'seismic')
        print(str(np.round(dif.bincenters[time_idx],3)))
        time_idx += 1
    

In [ ]:
fig, axes = plt.subplots(nrows = 7, ncols = 5, figsize = (6.496063, 9.094488), dpi =150)
time_idx = 0
for i in np.arange(7):
    for j in np.arange(5):
        im = axes[i,j].imshow(dif.alinged_imgs[time_idx] - dif.alinged_imgs[0:5].mean(axis = 0), vmin = -.025, vmax= .025, cmap = 'seismic')
        s = str(np.round(dif.bincenters[time_idx],3))
        axes[i,j].text( 500, 200, s + ' ps', ha='center')
#         i, dif.a_list[j][i]+0.003, label, fontsize=9, ha='center')
        time_idx += 1

        
        
        
        axes[i,j].set_xticks([]);
        axes[i,j].set_yticks([]);
        
# for ax in axes:
#     ax.set_xticks([]);
#     ax.set_yticks([]);
    
# fig.tick_params
# axes.set_xticks([]);
# axes.set_yticks([]);
# plt.tight_layout()

fig.savefig('full_diffuse_scattering_patterns_aligned.jpeg', dpi=300)

### Full images for non corrected diffuse

In [ ]:
fig, axes = plt.subplots(nrows = 7, ncols = 5, figsize = (6.496063, 9.094488), dpi =150)
time_idx = 0
for i in np.arange(7):
    for j in np.arange(5):
        im = axes[i,j].imshow(dif.binned_images[time_idx] - np.asarray(dif.binned_images)[0:5].mean(axis = 0), vmin = -.025, vmax= .025, cmap = 'seismic')
        s = str(np.round(dif.bincenters[time_idx],3))
        axes[i,j].text( 500, 200, s + ' ps', ha='center')
#         i, dif.a_list[j][i]+0.003, label, fontsize=9, ha='center')
        time_idx += 1

        
        
        
        axes[i,j].set_xticks([]);
        axes[i,j].set_yticks([]);
        
# for ax in axes:
#     ax.set_xticks([]);
#     ax.set_yticks([]);
    
# fig.tick_params
# axes.set_xticks([]);
# axes.set_yticks([]);
# plt.tight_layout()

fig.savefig('full_diffuse_scattering_patterns_NOT_aligned.jpeg', dpi=300)

In [ ]:
# Show final results
idx = 18

fig, ax2 = plt.subplots(ncols= 1, figsize = (6,5), dpi = 300)
im2 = ax2.imshow(dif.binned_images[idx] - dif.binned_images[0], vmin = -0.025, vmax= +0.025, cmap = 'seismic')
ax2.set_title(f'NOT SCALED Diffuse, {np.round(dif.bincenters[idx], 4)} ps')
ax2.set_xticks([]);
ax2.set_yticks([]);
plt.colorbar(im2, ax=ax2)


## Get linecuts 

In [ ]:

# h = 2
# k = 0

A,B,C,D,E,F,G,H = I_UED_dt.select_braggs_Bragg(h=h, k=k, allBragg_indices_2=dif.allBragg_indices_2)
# A_M,B_M,C_M,D_M,E_M,F_M,G_M,H_M = select_braggs_Bragg(h=h, k=k, allBragg_indices_2=allBragg_indices_2)

In [ ]:
A,B,C,D,E,F,G,H = I_UED_dt.select_braggs_Bragg(h=h, k=k, allBragg_indices_2=dif.allBragg_indices_2)
# A_M,B_M,C_M,D_M,E_M,F_M,G_M,H_M = select_braggs_Bragg(h=h, k=k, allBragg_indices_2=allBragg_indices_2)

### Use Bragg spots to take G-M and G-X paths

In [ ]:
h2 = 3
k2 = 0
A2,B2,C2,D2,E2,F2,G2,H2 = I_UED_dt.select_braggs_Bragg(h=h2, k=k2, allBragg_indices_2=dif.allBragg_indices_2)

In [ ]:
h3 = h2+0
k3 = k2+1
A3,B3,C3,D3,E3,F3,G3,H3 = I_UED_dt.select_braggs_Bragg(h=h3, k=k3, allBragg_indices_2=dif.allBragg_indices_2)

In [ ]:
def plot_rectangles (A2, A3, color = 'red', ls='-', size=int(dif.bz_len/10)):
    poly,poly_points = ued_dt.rect_profile(*dif.bragg_fits_binned_delays[0,A2][1:3],*dif.bragg_fits_binned_delays[0,A3][1:3],size)
    ax.plot([poly_points[0][1], poly_points[0][2]], [poly_points[1][1], poly_points[1][2]], color = color, ls = ls)
    ax.plot([poly_points[0][2], poly_points[0][3]], [poly_points[1][2], poly_points[1][3]], color = color, ls = ls)
    ax.plot([poly_points[0][3], poly_points[0][0]], [poly_points[1][3], poly_points[1][0]], color = color, ls = ls)
    ax.plot([poly_points[0][0], poly_points[0][1]], [poly_points[1][0], poly_points[1][1]], color = color, ls = ls)

    

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(image - dif.bkg_img_bkg, vmin = -0, vmax = 10,)

ax.scatter(dif.bragg_fits_binned_delays[0,A2][1], dif.bragg_fits_binned_delays[0,A2][2], color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,A3][1], dif.bragg_fits_binned_delays[0,A3][2], color = 'red')
plot_rectangles (A2=A2, A3=A3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,B2][1], dif.bragg_fits_binned_delays[0,B2][2], color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,B3][1], dif.bragg_fits_binned_delays[0,B3][2], color = 'red')
plot_rectangles (A2=B2, A3=B3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,C2][1], dif.bragg_fits_binned_delays[0,C2][2], color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,C3][1], dif.bragg_fits_binned_delays[0,C3][2], color = 'red')
plot_rectangles (A2=C2, A3=C3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,D2][1], dif.bragg_fits_binned_delays[0,D2][2], color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,D3][1], dif.bragg_fits_binned_delays[0,D3][2], color = 'red')
plot_rectangles (A2=D2, A3=D3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,E2][1], dif.bragg_fits_binned_delays[0,E2][2], )#color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,E3][1], dif.bragg_fits_binned_delays[0,E3][2], )#color = 'red')
plot_rectangles (A2=E2, A3=E3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,F2][1], dif.bragg_fits_binned_delays[0,F2][2], )#color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,F3][1], dif.bragg_fits_binned_delays[0,F3][2], )#color = 'red')
plot_rectangles (A2=F2, A3=F3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,G2][1], dif.bragg_fits_binned_delays[0,G2][2], )#color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,G3][1], dif.bragg_fits_binned_delays[0,G3][2], )#color = 'red')
plot_rectangles (A2=G2, A3=G3, color = 'white')

ax.scatter(dif.bragg_fits_binned_delays[0,H2][1], dif.bragg_fits_binned_delays[0,H2][2], )#color = 'white')
ax.scatter(dif.bragg_fits_binned_delays[0,H3][1], dif.bragg_fits_binned_delays[0,H3][2], )#color = 'red')
plot_rectangles (A2=H2, A3=H3, color = 'white')


In [ ]:
### First we get the corrdinate of all bragg spots

### First we get the corrdinate of all bragg spots

In [ ]:
eq_linecut_idxs = [[A2,A3], [B2,B3], [C2,C3], [D2,D3],[E2,E3],[F2,F3],[G2,G3],[H2,H3]]
# eq_linecut_idxs = [[A2,A3], [B2,B3],[E2,E3],[F2,F3],[G2,G3],[H2,H3]] #this one works for 

for i, letter in enumerate(eq_linecut_idxs):
    print(i, letter)

In [ ]:
line_profiles = dict()
profile_length = 30
line_profiles_intrp = np.zeros((len(dif.bincenters), 8, profile_length))

for t_idx in np.arange(len(dif.bincenters)):    
#     print(t_idx)
    temp_dict = dict()
    
    image = dif.binned_images[t_idx]#- empty_img2
    for i, letter in enumerate(eq_linecut_idxs):
        profile = skimage.measure.profile_line(image.T,
                                           dif.bragg_fits_binned_delays[t_idx,letter[0]][1:3],
                                           dif.bragg_fits_binned_delays[t_idx,letter[1]][1:3],
                                           linewidth =  15#int(bz_len/10)
                                               , order=0)
        
#         print(len(profile))
        # bin profile to the selected number of points: 
        profile_coords = np.arange(len(profile))
        new_x = 100*np.arange(profile_length)/profile_length
        indx_list = np.digitize(profile_coords, new_x)-1

        profile_intrp = np.zeros(profile_length)
        for j, idx in enumerate(indx_list):
            profile_intrp[idx] += profile[j]/np.sum(indx_list == idx)

        line_profiles_intrp[t_idx,i] = profile_intrp

        
    line_profiles[t_idx] = profile
    
    


In [ ]:
cmap = plt.get_cmap('rainbow')

fig, ax = plt.subplots(2, figsize = (8,8), sharex = True)
for t_idx in np.arange(len(dif.bincenters)-0)+0:
    color = cmap(float(t_idx-0)/(len(dif.a_list)-0))
#     print(line_profiles[t_idx].shape)
    ax[1].plot(new_x, line_profiles_intrp[t_idx,3] - line_profiles_intrp[1,3], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[0].plot(new_x, line_profiles_intrp[t_idx,3],# - line_profiles_intrp[0,3], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )

# plt.ylim(-0.1/2,0.1/2)
ax[0].set_xlim(0,100)
ax[1].set_ylim(-0.06,0.06)



ax[0].legend(ncol = 5,fontsize = 8 )
ax[0].set_ylabel('I (arb. units)')

ax[1].set_ylabel('I - I0 (arb. units)')


In [ ]:
symetrised_profile = np.mean(line_profiles_intrp, axis = 1)
symetrised_profile_t0 = np.mean(symetrised_profile[0:6] ,axis = 0)
symetrised_profile_t0.shape

In [ ]:
fig, ax = plt.subplots( figsize = (8,8), sharex = True)
for t_idx in np.arange(4, len(dif.bincenters),3)-4:

    color = cmap(float(t_idx)/(len(dif.a_list)-5))
    
    ax.plot(new_x, 0.00*t_idx + symetrised_profile[t_idx] - symetrised_profile[1], 
            color = color, label = f'{np.round(dif.bincenters[t_idx],3)} ps' )

ax.legend(ncol = 2,fontsize = 8 , frameon = False)
ax.set_xlim([0,100])
ax.set_ylim([-0.01,0.025])

ax.set_ylabel('I - I0 (arb. units)')
ax.axvline(50,ls = '--', c='k',alpha = 0.5)

ax.set_xticks([0, 50, 100]);
ax.set_xticklabels([r'$\Gamma_{300}$',r'$X_T$', r'$\Gamma_{310}$',], fontsize = 16)
print((h2, k2))
print((h3, k3))

In [ ]:
fig, ax = plt.subplots(4, figsize = (8,8), sharex = True)
for t_idx in np.arange(len(dif.bincenters)):
    color = cmap(float(t_idx)/len(dif.a_list))
#     print(line_profiles[t_idx].shape)
    
    ax[0].plot(new_x, line_profiles_intrp[t_idx,0] - line_profiles_intrp[0,0], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[1].plot(new_x, line_profiles_intrp[t_idx,1] - line_profiles_intrp[0,1], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[2].plot(new_x, line_profiles_intrp[t_idx,2] - line_profiles_intrp[0,2], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[3].plot(new_x, line_profiles_intrp[t_idx,3] - line_profiles_intrp[0,3], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )


# plt.ylim(-0.1/2,0.1/2)
ax[0].set_xlim(0,100)
ax[0].set_ylim(-0.06/2,0.06)

ax[1].set_xlim(0,100)
ax[1].set_ylim(-0.06/2,0.06)
ax[2].set_xlim(0,100)
ax[2].set_ylim(-0.06/2,0.06)
ax[3].set_xlim(0,100)
ax[3].set_ylim(-0.06/2,0.06)

# ax[0].legend(ncol = 5,fontsize = 8 )
ax[0].set_ylabel('I (arb. units)')

ax[1].set_ylabel('I - I0 (arb. units)')


In [ ]:
fig, ax = plt.subplots(4, figsize = (8,8), sharex = True)
for t_idx in np.arange(len(dif.bincenters)):
    color = cmap(float(t_idx)/len(dif.a_list))
#     print(line_profiles[t_idx].shape)
    
    ax[0].plot(new_x, line_profiles_intrp[t_idx,4] - line_profiles_intrp[0,4], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[1].plot(new_x, line_profiles_intrp[t_idx,1+4] - line_profiles_intrp[0,1+4], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[2].plot(new_x, line_profiles_intrp[t_idx,2+4] - line_profiles_intrp[0,2+4], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )
    
    ax[3].plot(new_x, line_profiles_intrp[t_idx,3+4] - line_profiles_intrp[0,3+4], 
             color = color, label = np.round(dif.bincenters[t_idx],3) )


# plt.ylim(-0.1/2,0.1/2)
ax[0].set_xlim(0,100)
ax[0].set_ylim(-0.06,0.06)

ax[1].set_xlim(0,100)
ax[1].set_ylim(-0.06,0.06)
ax[2].set_xlim(0,100)
ax[2].set_ylim(-0.06,0.06)
ax[3].set_xlim(0,100)
ax[3].set_ylim(-0.06,0.06)

# ax[0].legend(ncol = 5,fontsize = 8 )
ax[0].set_ylabel('I (arb. units)')

ax[1].set_ylabel('I - I0 (arb. units)')


In [ ]:
def norm_tmp(profile):
    subtracted = profile /profile[0:4].mean()
    return subtracted

In [ ]:
def subt0_tmp(profile):
    subtracted = profile -profile[0:4].mean()
    return subtracted

## PLOTS FOR DIFFERENT BRAGG BZ

### 100 to (110, 200, 210)

In [ ]:
fig, ax = plt.subplots(figsize = (8/1.2,8/1.2))
ax.pcolormesh(image - dif.bkg_img_bkg, vmin = -0, vmax = 10,)


h2 = 4
k2 = 0
A2,B2,C2,D2,E2,F2,G2,H2 = I_UED_dt.select_braggs_Bragg(h=h2, k=k2, allBragg_indices_2=dif.allBragg_indices_2)

h3 = h2+0
k3 = k2+1
A3,B3,C3,D3,E3,F3,G3,H3 = I_UED_dt.select_braggs_Bragg(h=h3, k=k3, allBragg_indices_2=dif.allBragg_indices_2)


plot_rectangles (A2=A2, A3=A3, color = 'k')
plot_rectangles (A2=B2, A3=B3, color = 'k')
plot_rectangles (A2=C2, A3=C3, color = 'k')
plot_rectangles (A2=D2, A3=D3, color = 'k')
plot_rectangles (A2=E2, A3=E3, color = 'k')
plot_rectangles (A2=F2, A3=F3, color = 'k')
plot_rectangles (A2=G2, A3=G3, color = 'k')
plot_rectangles (A2=H2, A3=H3, color = 'k')



h3 = h2+1
k3 = k2+0
A3,B3,C3,D3,E3,F3,G3,H3 = I_UED_dt.select_braggs_Bragg(h=h3, k=k3, allBragg_indices_2=dif.allBragg_indices_2)


plot_rectangles (A2=A2, A3=A3, color = 'blue')
plot_rectangles (A2=B2, A3=B3, color = 'blue')
plot_rectangles (A2=C2, A3=C3, color = 'blue')
plot_rectangles (A2=D2, A3=D3, color = 'blue')
plot_rectangles (A2=E2, A3=E3, color = 'blue')
plot_rectangles (A2=F2, A3=F3, color = 'blue')
plot_rectangles (A2=G2, A3=G3, color = 'blue')
plot_rectangles (A2=H2, A3=H3, color = 'blue')



## Here is the q dot k

In [ ]:
# def profile_func(test_A, dif, retrun_intrp = False, profile_length = 100, linewidth=15):
def profile_func(test_A, dif, retrun_intrp = False, profile_length = 100, linewidth=int(dif.bz_len*20/100)):

    
    '''
    Arguments:
    test_A : List of indexes, the indexes should correspond to the bragg spot
    return_intrp : if ==True always returns 100 point arrays
    
    Retruns:
    dictionary with the profiles 
    if return_intrp == True the profiles have always 100 point length
    '''

    line_profiles = dict()
#     profile_length = 100
    line_profiles_intrp = np.zeros((len(dif.bincenters), len(test_A), profile_length))

    
    for t_idx in np.arange(len(dif.bincenters)):
        image = dif.binned_images[t_idx]#- empty_img2
        for i, letter in enumerate(test_A):
            profile = skimage.measure.profile_line(image.T,
                                               dif.bragg_fits_binned_delays[t_idx,letter[0]][1:3],
                                               dif.bragg_fits_binned_delays[t_idx,letter[1]][1:3],
                                               linewidth =  linewidth#int(bz_len/10)
                                                   , order=0)

            profile_coords = np.arange(len(profile))
            new_x = 100*np.arange(profile_length)/profile_length
            indx_list = np.digitize(profile_coords, new_x)-1

            profile_intrp = np.zeros(profile_length)
            for j, idx in enumerate(indx_list):
                profile_intrp[idx] += profile[j]/np.sum(indx_list == idx)
            line_profiles_intrp[t_idx,i] = profile_intrp
        line_profiles[t_idx] = profile
    
    
    if retrun_intrp == True:
        return line_profiles_intrp
    else:
        return line_profiles

    

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
im =ax.imshow(image - dif.bkg_img_bkg, vmin = -0.25, vmax = 1,cmap = 'binary')
plt.scatter (dif.allBragg_coord_2[:,0], dif.allBragg_coord_2[:,1], s = 2)

plt.xlim(0,1024)
plt.ylim(0,1024)
plt.colorbar(im)

In [ ]:
def plot_rectangles_ROI (A2, A3, color = 'red', length= 10, width=10 ):#int(bz_len/10)):
    '''
    I don't rebember where I got this function from,  probably Igor's 
    '''
    pos_bragg_A2 = dif.bragg_fits_binned_delays[0,A2][1:3]
    pos_bragg_A3 = dif.bragg_fits_binned_delays[0,A3][1:3]
    diff = pos_bragg_A3 - pos_bragg_A2
    
#     bz_len = np.linalg.norm(diff)/2
#     start_pos = pos_bragg_A2+(diff/2)*(1-0.2)
#     end_pos = pos_bragg_A3-(diff/2)*(1-0.2)
    
    start_pos = pos_bragg_A2+(diff/2) - 0.5*length*diff/np.linalg.norm(diff)
    end_pos = pos_bragg_A3-(diff/2) + 0.5*length*diff/np.linalg.norm(diff)
    
#     print(dif.bragg_fits_binned_delays[0,A2][1:3])
    poly,poly_points = ued_dt.rect_profile(*start_pos,*end_pos, width = width)
    
    ax.plot([poly_points[0][1], poly_points[0][2]], [poly_points[1][1], poly_points[1][2]], color = color)
    ax.plot([poly_points[0][2], poly_points[0][3]], [poly_points[1][2], poly_points[1][3]], color = color)
    ax.plot([poly_points[0][3], poly_points[0][0]], [poly_points[1][3], poly_points[1][0]], color = color)
    ax.plot([poly_points[0][0], poly_points[0][1]], [poly_points[1][0], poly_points[1][1]], color = color)

In [ ]:
def get_rectangles_ROI_fill (A2, A3, color = 'red', length= 10, width=10 ):#int(bz_len/10)):
    '''
    I don't rebember where I got this function from,  probably Igor's 
    '''
    
    from matplotlib.path import Path
    import matplotlib.patches as patches
    
    pos_bragg_A2 = dif.bragg_fits_binned_delays[0,A2][1:3]
    pos_bragg_A3 = dif.bragg_fits_binned_delays[0,A3][1:3]
    diff = pos_bragg_A3 - pos_bragg_A2
        
#     start_pos = pos_bragg_A2+(diff/2)*(1-0.2)
#     end_pos = pos_bragg_A3-(diff/2)*(1-0.2)
    
    start_pos = pos_bragg_A2+(diff/2) - 0.5*length*diff/np.linalg.norm(diff)
    end_pos = pos_bragg_A3-(diff/2) + 0.5*length*diff/np.linalg.norm(diff)    

#     print(dif.bragg_fits_binned_delays[0,A2][1:3])
    poly,poly_points = ued_dt.rect_profile(*start_pos,*end_pos, width = width)
    
    
    print(np.asarray(poly_points)[:,0].shape )
    
    
    verts = [np.asarray(poly_points)[:,0], # left, bottom
         np.asarray(poly_points)[:,1], # left, top
         np.asarray(poly_points)[:,2], # right, top
         np.asarray(poly_points)[:,3], # right, bottom
         np.asarray(poly_points)[:,0],] # ignored
    
    codes = [Path.MOVETO,
         Path.LINETO,
         Path.LINETO,
         Path.LINETO,
         Path.CLOSEPOLY,]
    path = Path(verts, codes)
    patch = patches.PathPatch(path, facecolor=color, lw=0, alpha = 0.5)
    
    return patch #= patches.PathPatch(path, facecolor=color, lw=0, alpha = 0.5)
    
#     y_idx_max = x
#     print(np.argsort(np.asarray(poly_points)[1]) )
#     top_y = np.argsort()
#     ax.plot([poly_points[0][1], poly_points[0][2]], [poly_points[1][1], poly_points[1][2]], color = color)
#     ax.plot([poly_points[0][2], poly_points[0][3]], [poly_points[1][2], poly_points[1][3]], color = color)
#     ax.plot([poly_points[0][3], poly_points[0][0]], [poly_points[1][3], poly_points[1][0]], color = color)
#     ax.plot([poly_points[0][0], poly_points[0][1]], [poly_points[1][0], poly_points[1][1]], color = color)

In [ ]:
dif.BZ_portion = 20
dif.bz_ROI_width = 15

fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(image - dif.bkg_img_bkg, vmin = -.1, vmax = 3,cmap = 'binary')

dif.diffuse_profiles = dif.Static()

dif.diffuse_profiles.big_profile_dict_2 = dict()
dif.diffuse_profiles.big_profile_dict_3 = dict()

dif.diffuse_profiles.big_profile_dict_GM = dict()

dif.diffuse_profiles.big_X_pos_dict_2 = dict()
dif.diffuse_profiles.big_X_pos_dict_3 = dict()

dif.diffuse_profiles.big_X_pos_dict_time_2 = dict()
dif.diffuse_profiles.big_X_pos_dict_time_3 = dict()
dif.diffuse_profiles.big_M_pos_dict_time= dict()

dif.diffuse_profiles.big_M_pos_dict = dict()

dif.diffuse_profiles.big_X_pos_dict_2_20pc = dict()
dif.diffuse_profiles.big_X_pos_dict_3_20pc = dict()
dif.diffuse_profiles.big_X_pos_dict_2_60pc = dict()
dif.diffuse_profiles.big_X_pos_dict_3_60pc = dict()

dif.diffuse_profiles.big_M_pos_dict_20pc = dict()
dif.diffuse_profiles.big_M_pos_dict_60pc = dict()

dif.diffuse_profiles.big_A1_pos_dict = dict()
dif.diffuse_profiles.big_A2_pos_dict = dict()
dif.diffuse_profiles.big_A3_pos_dict = dict()
dif.diffuse_profiles.big_A4_pos_dict = dict()

dif.diffuse_profiles.big_A1_pos_t_dict = dict()
dif.diffuse_profiles.big_A2_pos_t_dict = dict()
dif.diffuse_profiles.big_A3_pos_t_dict = dict()
dif.diffuse_profiles.big_A4_pos_t_dict = dict()


already_start_point = np.asarray([])


iter_list = [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [3, 0], [3, 1], [3, 2],[4, 0]]
# iter_list = [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [3, 0], [3, 1], [4, 0]]

for h,k in iter_list:
    print(h,k)
    
    
# for h in range(1,4):
#     for k in range(3):
#         print(h, k)

    A1,B1,C1,D1,E1,F1,G1,H1 = I_UED_dt.select_braggs_Bragg(h=h,   k=k,   allBragg_indices_2=dif.allBragg_indices_2)
    A2,B2,C2,D2,E2,F2,G2,H2 = I_UED_dt.select_braggs_Bragg(h=h+1, k=k,   allBragg_indices_2=dif.allBragg_indices_2)
    A3,B3,C3,D3,E3,F3,G3,H3 = I_UED_dt.select_braggs_Bragg(h=h,   k=k+1, allBragg_indices_2=dif.allBragg_indices_2)
    
    A4,B4,C4,D4,E4,F4,G4,H4 = I_UED_dt.select_braggs_Bragg(h=h+1,   k=k+1, allBragg_indices_2=dif.allBragg_indices_2)

    test = np.asarray([A1,B1,C1,D1,E1,F1,G1,H1])
    if ((A1 not in already_start_point)and(B1 not in already_start_point)
        and(C1 not in already_start_point)and(D1 not in already_start_point)
        and(E1 not in already_start_point)and(F1 not in already_start_point)
        and(G1 not in already_start_point)and(H1 not in already_start_point)):

        already_start_point= np.append(already_start_point,test )

        test_A = [[A1,A2], [B1,B2], [C1,C2], [D1,D2],[E1,E2],[F1,F2],[G1,G2],[H1,H2]]
        dif.diffuse_profiles.big_profile_dict_2[h,k] = profile_func(test_A, dif, linewidth=int(dif.bz_len*dif.BZ_portion/100), retrun_intrp = True)

        test_A2 = [[A1,A3], [B1,B3], [C1,C3], [D1,D3],[E1,E3],[F1,F3],[G1,G3],[H1,H3]]
        dif.diffuse_profiles.big_profile_dict_3[h,k] = profile_func(test_A2, dif, linewidth=int(dif.bz_len*dif.BZ_portion/100), retrun_intrp = True)

        test_M = [[A1,A4], [B1,B4], [C1,C4], [D1,D4],[E1,E4],[F1,F4],[G1,G4],[H1,H4]]
        dif.diffuse_profiles.big_profile_dict_GM[h,k] = profile_func(test_M, dif, linewidth=int(dif.bz_len*dif.BZ_portion/100), retrun_intrp = True)
        dif.diffuse_profiles.big_profile_dict_GM[h,k][:,np.asarray(np.where(~dif.sel_bragg_in_im[np.asarray(test_M)[:,1]] ))] = np.nan
        
#         #### Too problematic, won't implement
#         if (bragg_fits_binned_delays[0,[A2,B2,C2,D2,E2,F2,G2,H2], 1:3] == 0.).any():
#             print('fuck')
#         else:

        dif.diffuse_profiles.big_X_pos_dict_2[h+1,k] = 0.5*(dif.bragg_fits_binned_delays[0,A1][1:3] + dif.bragg_fits_binned_delays[0,A2][1:3])
        dif.diffuse_profiles.big_X_pos_dict_3[h,k+1] = 0.5*(dif.bragg_fits_binned_delays[0,A1][1:3] + dif.bragg_fits_binned_delays[0,A3][1:3])
        dif.diffuse_profiles.big_M_pos_dict[h+1,k+1] = 0.5*(dif.bragg_fits_binned_delays[0,A1][1:3] + dif.bragg_fits_binned_delays[0,A4][1:3])
        
        dif.diffuse_profiles.big_X_pos_dict_time_2[h+1,k] = 0.5*(dif.bragg_fits_binned_delays[:,A1,1:3] + dif.bragg_fits_binned_delays[:,A2,1:3])
        dif.diffuse_profiles.big_X_pos_dict_time_3[h,k+1] = 0.5*(dif.bragg_fits_binned_delays[:,A1,1:3] + dif.bragg_fits_binned_delays[:,A3,1:3])
        dif.diffuse_profiles.big_M_pos_dict_time[h+1,k+1] = 0.5*(dif.bragg_fits_binned_delays[0,A1,1:3] + dif.bragg_fits_binned_delays[0,A4,1:3])

        dif.diffuse_profiles.big_X_pos_dict_2_20pc[h+1,k] = (0.3*dif.bragg_fits_binned_delays[0,A1][1:3] + 0.7*dif.bragg_fits_binned_delays[0,A2][1:3])
        dif.diffuse_profiles.big_X_pos_dict_3_20pc[h,k+1] = (0.3*dif.bragg_fits_binned_delays[0,A1][1:3] + 0.7*dif.bragg_fits_binned_delays[0,A3][1:3])
        dif.diffuse_profiles.big_X_pos_dict_2_60pc[h+1,k] = (0.7*dif.bragg_fits_binned_delays[0,A1][1:3] + 0.3*dif.bragg_fits_binned_delays[0,A2][1:3])
        dif.diffuse_profiles.big_X_pos_dict_3_60pc[h,k+1] = (0.7*dif.bragg_fits_binned_delays[0,A1][1:3] + 0.3*dif.bragg_fits_binned_delays[0,A3][1:3])
        
        dif.diffuse_profiles.big_M_pos_dict_20pc[h+1,k+1] = (0.3*dif.bragg_fits_binned_delays[0,A1][1:3] + 0.7*dif.bragg_fits_binned_delays[0,A4][1:3])
        dif.diffuse_profiles.big_M_pos_dict_60pc[h+1,k+1] = (0.7*dif.bragg_fits_binned_delays[0,A1][1:3] + 0.3*dif.bragg_fits_binned_delays[0,A4][1:3])        
        
        
        dif.diffuse_profiles.big_A1_pos_dict[h,k]   = dif.bragg_fits_binned_delays[0,A1][1:3]
        dif.diffuse_profiles.big_A2_pos_dict[h+1,k] = dif.bragg_fits_binned_delays[0,A2][1:3]
        dif.diffuse_profiles.big_A3_pos_dict[h,k+1] = dif.bragg_fits_binned_delays[0,A3][1:3]
        dif.diffuse_profiles.big_A4_pos_dict[h+1,k+1] = dif.bragg_fits_binned_delays[0,A4][1:3]
        
        dif.diffuse_profiles.big_A1_pos_t_dict[h,k]     = dif.bragg_fits_binned_delays[:,A1,1:3]
        dif.diffuse_profiles.big_A2_pos_t_dict[h+1,k]   = dif.bragg_fits_binned_delays[:,A2,1:3]
        dif.diffuse_profiles.big_A3_pos_t_dict[h,k+1]   = dif.bragg_fits_binned_delays[:,A3,1:3]
        dif.diffuse_profiles.big_A4_pos_t_dict[h+1,k+1] = dif.bragg_fits_binned_delays[:,A4,1:3]
        
        
        
        ax.scatter(dif.bragg_fits_binned_delays[0,A2][1], dif.bragg_fits_binned_delays[0,A2][2], color = 'blue')
        ax.scatter(dif.bragg_fits_binned_delays[0,A3][1], dif.bragg_fits_binned_delays[0,A3][2], color = 'blue')
#         plot_rectangles (A2=A1, A3=A3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=A1, A3=A2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=A1, A3=A3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width  )
        plot_rectangles_ROI (A2=A1, A3=A2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width  )
        if dif.sel_bragg_in_im[A4]:
#             plot_rectangles (A2=A1, A3=A4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=A1, A3=A4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width)
            

        ax.scatter(dif.bragg_fits_binned_delays[0,B2][1], dif.bragg_fits_binned_delays[0,B2][2], color = 'blue')
        ax.scatter(dif.bragg_fits_binned_delays[0,B3][1], dif.bragg_fits_binned_delays[0,B3][2], color = 'blue')
#         plot_rectangles (A2=B1, A3=B3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=B1, A3=B2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=B1, A3=B3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=B1, A3=B2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )        
        if dif.sel_bragg_in_im[B4]:
#             plot_rectangles (A2=B1, ls = '--', A3=B4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=B1, A3=B4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

        ax.scatter(dif.bragg_fits_binned_delays[0,C2][1], dif.bragg_fits_binned_delays[0,C2][2], color = 'blue')
        ax.scatter(dif.bragg_fits_binned_delays[0,C3][1], dif.bragg_fits_binned_delays[0,C3][2], color = 'blue')
#         plot_rectangles (A2=C1, A3=C3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=C1, A3=C2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=C1, A3=C3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=C1, A3=C2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[C4]:
#             plot_rectangles (A2=C1, A3=C4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=C1, A3=C4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            
            
        ax.scatter(dif.bragg_fits_binned_delays[0,D2][1], dif.bragg_fits_binned_delays[0,D2][2], color = 'blue')
        ax.scatter(dif.bragg_fits_binned_delays[0,D3][1], dif.bragg_fits_binned_delays[0,D3][2], color = 'blue')
#         plot_rectangles (A2=D1, A3=D3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=D1, A3=D2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=D1, A3=D3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=D1, A3=D2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )        
        if dif.sel_bragg_in_im[D4]:
#             plot_rectangles (A2=D1, A3=D4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=D1, A3=D4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

        ax.scatter(dif.bragg_fits_binned_delays[0,E2][1], dif.bragg_fits_binned_delays[0,E2][2], )#color = 'white')
        ax.scatter(dif.bragg_fits_binned_delays[0,E3][1], dif.bragg_fits_binned_delays[0,E3][2], )#color = 'red')
#         plot_rectangles (A2=E1, A3=E3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=E1, A3=E2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=E1, A3=E3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=E1, A3=E2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[E4]:
#             plot_rectangles (A2=E1, A3=E4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=E1, A3=E4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

        ax.scatter(dif.bragg_fits_binned_delays[0,F2][1], dif.bragg_fits_binned_delays[0,F2][2], )#color = 'white')
        ax.scatter(dif.bragg_fits_binned_delays[0,F3][1], dif.bragg_fits_binned_delays[0,F3][2], )#color = 'red')
#         plot_rectangles (A2=F1, A3=F3, ls = '--', color = 'blue', size = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=F1, A3=F2, ls = '--', color = 'blue', size = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=F1, A3=F3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=F1, A3=F2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[G4]:
#             plot_rectangles (A2=F1, A3=F4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=F1, A3=F4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            
            
        ax.scatter(dif.bragg_fits_binned_delays[0,G2][1], dif.bragg_fits_binned_delays[0,G2][2], )#color = 'white')
        ax.scatter(dif.bragg_fits_binned_delays[0,G3][1], dif.bragg_fits_binned_delays[0,G3][2], )#color = 'red')
#         plot_rectangles (A2=G1, A3=G3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=G1, A3=G2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=G1, A3=G3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=G1, A3=G2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[G4]:
#             plot_rectangles (A2=G1, A3=G4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=G1, A3=G4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

        ax.scatter(dif.bragg_fits_binned_delays[0,H2][1], dif.bragg_fits_binned_delays[0,H2][2], )#color = 'white')
        ax.scatter(dif.bragg_fits_binned_delays[0,H3][1], dif.bragg_fits_binned_delays[0,H3][2], )#color = 'red')
#         plot_rectangles (A2=H1, A3=H3, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
#         plot_rectangles (A2=H1, A3=H2, ls = '--', color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100))
        plot_rectangles_ROI (A2=H1, A3=H3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=H1, A3=H2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[H4]:
#             plot_rectangles (A2=H1, A3=H4, ls = '--', color = 'green', length = int(dif.bz_len*dif.BZ_portion/100))
            plot_rectangles_ROI (A2=H1, A3=H4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )

# for i in np.arange(len(list(dif.diffuse_profiles.big_X_pos_dict_3.keys()))):
#     key2_tmp = list(dif.diffuse_profiles.big_X_pos_dict_2.keys())[i]
#     key3_tmp = list(dif.diffuse_profiles.big_X_pos_dict_3.keys())[i]
#     keyM_tmp = list(dif.diffuse_profiles.big_M_pos_dict.keys())[i]
    
#     plt.scatter(dif.diffuse_profiles.big_X_pos_dict_2[key2_tmp][0], dif.diffuse_profiles.big_X_pos_dict_2[key2_tmp][1], marker='x')
#     plt.scatter(dif.diffuse_profiles.big_X_pos_dict_3[key3_tmp][0], dif.diffuse_profiles.big_X_pos_dict_3[key3_tmp][1], marker='x')
#     plt.scatter(dif.diffuse_profiles.big_M_pos_dict[keyM_tmp][0], dif.diffuse_profiles.big_M_pos_dict[keyM_tmp][1], marker='^')


# #         print(A1,B1,C1,D1,E1,F1,G1,H1)
# #         print(A1, A2, A3)

In [ ]:
dif.BZ_portion = 20
# dif.bz_ROI_width = 15
dif.bz_ROI_width = 20

# fig, ax = plt.subplots(figsize = (8,8), dpi=300)
fig, ax = plt.subplots(figsize = (8,8))

ax.imshow(image - dif.bkg_img_bkg, vmin = -.1, vmax = 3,cmap = 'binary')

already_start_point = np.asarray([])

iter_list = [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [3, 0], [3, 1], [3, 2],[4, 0]]
# iter_list = [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [3, 0], [3, 1], [4, 0]]

for h,k in iter_list:
    print(h,k)
    
    
# for h in range(1,4):
#     for k in range(3):
#         print(h, k)

    A1,B1,C1,D1,E1,F1,G1,H1 = I_UED_dt.select_braggs_Bragg(h=h,   k=k,   allBragg_indices_2=dif.allBragg_indices_2)
    A2,B2,C2,D2,E2,F2,G2,H2 = I_UED_dt.select_braggs_Bragg(h=h+1, k=k,   allBragg_indices_2=dif.allBragg_indices_2)
    A3,B3,C3,D3,E3,F3,G3,H3 = I_UED_dt.select_braggs_Bragg(h=h,   k=k+1, allBragg_indices_2=dif.allBragg_indices_2)
    
    A4,B4,C4,D4,E4,F4,G4,H4 = I_UED_dt.select_braggs_Bragg(h=h+1,   k=k+1, allBragg_indices_2=dif.allBragg_indices_2)

    test = np.asarray([A1,B1,C1,D1,E1,F1,G1,H1])
    if ((A1 not in already_start_point)and(B1 not in already_start_point)
        and(C1 not in already_start_point)and(D1 not in already_start_point)
        and(E1 not in already_start_point)and(F1 not in already_start_point)
        and(G1 not in already_start_point)and(H1 not in already_start_point)):

        already_start_point= np.append(already_start_point,test )

#         test_A = [[A1,A2], [B1,B2], [C1,C2], [D1,D2],[E1,E2],[F1,F2],[G1,G2],[H1,H2]]
#         test_A2 = [[A1,A3], [B1,B3], [C1,C3], [D1,D3],[E1,E3],[F1,F3],[G1,G3],[H1,H3]]
#         test_M = [[A1,A4], [B1,B4], [C1,C4], [D1,D4],[E1,E4],[F1,F4],[G1,G4],[H1,H4]]
        


#         ax.scatter(dif.bragg_fits_binned_delays[0,A2][1], dif.bragg_fits_binned_delays[0,A2][2], color = 'blue')
#         ax.scatter(dif.bragg_fits_binned_delays[0,A3][1], dif.bragg_fits_binned_delays[0,A3][2], color = 'blue')
#         plot_rectangles (A2=A1, A3=A3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=A1, A3=A2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=A1, A3=A3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width  )
        plot_rectangles_ROI (A2=A1, A3=A2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width  )
        if dif.sel_bragg_in_im[A4]:
#             plot_rectangles (A2=A1, A3=A4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=A1, A3=A4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width)
            

#         ax.scatter(dif.bragg_fits_binned_delays[0,B2][1], dif.bragg_fits_binned_delays[0,B2][2], color = 'blue')
#         ax.scatter(dif.bragg_fits_binned_delays[0,B3][1], dif.bragg_fits_binned_delays[0,B3][2], color = 'blue')
#         plot_rectangles (A2=B1, A3=B3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=B1, A3=B2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=B1, A3=B3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=B1, A3=B2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )        
        if dif.sel_bragg_in_im[B4]:
#             plot_rectangles (A2=B1, ls = '--', A3=B4, color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=B1, A3=B4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

#         ax.scatter(dif.bragg_fits_binned_delays[0,C2][1], dif.bragg_fits_binned_delays[0,C2][2], color = 'blue')
#         ax.scatter(dif.bragg_fits_binned_delays[0,C3][1], dif.bragg_fits_binned_delays[0,C3][2], color = 'blue')
#         plot_rectangles (A2=C1, A3=C3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=C1, A3=C2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=C1, A3=C3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=C1, A3=C2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[C4]:
#             plot_rectangles (A2=C1, A3=C4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=C1, A3=C4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            
            
#         ax.scatter(dif.bragg_fits_binned_delays[0,D2][1], dif.bragg_fits_binned_delays[0,D2][2], color = 'blue')
#         ax.scatter(dif.bragg_fits_binned_delays[0,D3][1], dif.bragg_fits_binned_delays[0,D3][2], color = 'blue')
#         plot_rectangles (A2=D1, A3=D3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=D1, A3=D2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=D1, A3=D3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=D1, A3=D2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )        
        if dif.sel_bragg_in_im[D4]:
#             plot_rectangles (A2=D1, A3=D4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=D1, A3=D4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

#         ax.scatter(dif.bragg_fits_binned_delays[0,E2][1], dif.bragg_fits_binned_delays[0,E2][2], )#color = 'white')
#         ax.scatter(dif.bragg_fits_binned_delays[0,E3][1], dif.bragg_fits_binned_delays[0,E3][2], )#color = 'red')
#         plot_rectangles (A2=E1, A3=E3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=E1, A3=E2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=E1, A3=E3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=E1, A3=E2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[E4]:
#             plot_rectangles (A2=E1, A3=E4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=E1, A3=E4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

#         ax.scatter(dif.bragg_fits_binned_delays[0,F2][1], dif.bragg_fits_binned_delays[0,F2][2], )#color = 'white')
#         ax.scatter(dif.bragg_fits_binned_delays[0,F3][1], dif.bragg_fits_binned_delays[0,F3][2], )#color = 'red')
#         plot_rectangles (A2=F1, A3=F3, ls = '--', color = 'blue', width = dif.bz_ROI_width)
#         plot_rectangles (A2=F1, A3=F2, ls = '--', color = 'blue', width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=F1, A3=F3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=F1, A3=F2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[G4]:
#             plot_rectangles (A2=F1, A3=F4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=F1, A3=F4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            
            
#         ax.scatter(dif.bragg_fits_binned_delays[0,G2][1], dif.bragg_fits_binned_delays[0,G2][2], )#color = 'white')
#         ax.scatter(dif.bragg_fits_binned_delays[0,G3][1], dif.bragg_fits_binned_delays[0,G3][2], )#color = 'red')
#         plot_rectangles (A2=G1, A3=G3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=G1, A3=G2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=G1, A3=G3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=G1, A3=G2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[G4]:
#             plot_rectangles (A2=G1, A3=G4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=G1, A3=G4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
            

#         ax.scatter(dif.bragg_fits_binned_delays[0,H2][1], dif.bragg_fits_binned_delays[0,H2][2], )#color = 'white')
#         ax.scatter(dif.bragg_fits_binned_delays[0,H3][1], dif.bragg_fits_binned_delays[0,H3][2], )#color = 'red')
#         plot_rectangles (A2=H1, A3=H3, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
#         plot_rectangles (A2=H1, A3=H2, ls = '--', color = 'blue',  width = dif.bz_ROI_width)
        plot_rectangles_ROI (A2=H1, A3=H3, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        plot_rectangles_ROI (A2=H1, A3=H2, color = 'blue', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )
        if dif.sel_bragg_in_im[H4]:
#             plot_rectangles (A2=H1, A3=H4, ls = '--', color = 'green',  width = dif.bz_ROI_width)
            plot_rectangles_ROI (A2=H1, A3=H4, color = 'green', length = int(dif.bz_len*dif.BZ_portion/100), width = dif.bz_ROI_width )

# for i in np.arange(len(list(dif.diffuse_profiles.big_X_pos_dict_3.keys()))):
#     key2_tmp = list(dif.diffuse_profiles.big_X_pos_dict_2.keys())[i]
#     key3_tmp = list(dif.diffuse_profiles.big_X_pos_dict_3.keys())[i]
#     keyM_tmp = list(dif.diffuse_profiles.big_M_pos_dict.keys())[i]
    
#     plt.scatter(dif.diffuse_profiles.big_X_pos_dict_2[key2_tmp][0], dif.diffuse_profiles.big_X_pos_dict_2[key2_tmp][1], marker='x')
#     plt.scatter(dif.diffuse_profiles.big_X_pos_dict_3[key3_tmp][0], dif.diffuse_profiles.big_X_pos_dict_3[key3_tmp][1], marker='x')
#     plt.scatter(dif.diffuse_profiles.big_M_pos_dict[keyM_tmp][0], dif.diffuse_profiles.big_M_pos_dict[keyM_tmp][1], marker='^')


# #         print(A1,B1,C1,D1,E1,F1,G1,H1)
# #         print(A1, A2, A3)

ax.set_xticks([]);
ax.set_yticks([]);

fig.savefig('Diffuse_X_M_ROIS_full.png', dpi=300)

In [ ]:
print(dif.bragg_fits_binned_delays[0,A4][1:3])
print(dif.bragg_fits_binned_delays[0,B4][1:3])
# print(dif.bragg_fits_binned_delays[0,C4][1:3])
# print(dif.bragg_fits_binned_delays[0,D4][1:3])
print(dif.bragg_fits_binned_delays[0,E4][1:3])
print(dif.bragg_fits_binned_delays[0,F4][1:3])
print(dif.bragg_fits_binned_delays[0,G4][1:3])
print(dif.bragg_fits_binned_delays[0,H4][1:3])

In [ ]:
print(list(dif.diffuse_profiles.big_A1_pos_dict.keys()))
print(list(dif.diffuse_profiles.big_A2_pos_dict.keys()))
print(list(dif.diffuse_profiles.big_A3_pos_dict.keys()))

In [ ]:
dif.diffuse_profiles.L_projection_X2 = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_X3 = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_M1 = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))

dif.diffuse_profiles.L_projection_X2_20pc = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_X3_20pc = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_X2_60pc = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_X3_60pc = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_M1_20pc = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))
dif.diffuse_profiles.L_projection_M1_60pc = np.zeros(len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ))


dif.diffuse_profiles.q_X2 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_X3 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_M1 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))

dif.diffuse_profiles.q_t_X2 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 2))
dif.diffuse_profiles.q_t_X3 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 2))
dif.diffuse_profiles.q_t_M1 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 2))

dif.diffuse_profiles.k_X2 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.k_X3 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.k_M1 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))

dif.diffuse_profiles.k_t_X2 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 2))
dif.diffuse_profiles.k_t_X3 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 2))
dif.diffuse_profiles.k_t_M1 =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 2))

dif.diffuse_profiles.q_X2_20pc =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_X3_20pc =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_X2_60pc =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_X3_60pc =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_M1_20pc =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))
dif.diffuse_profiles.q_M1_60pc =  np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 2))

dif.diffuse_profiles.I_X2_profile = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 100))
dif.diffuse_profiles.I_X3_profile = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 100))
dif.diffuse_profiles.I_M1_profile = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35, 100))


dif.diffuse_profiles.I_X2 = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_X3 = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_M1 = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))

dif.diffuse_profiles.I_X2_20pc = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_X3_20pc = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_X2_60pc = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_X3_60pc = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_M1_20pc = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))
dif.diffuse_profiles.I_M1_60pc = np.zeros((len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ), 35))

dif.diffuse_profiles.bragg_paths2 = []
dif.diffuse_profiles.bragg_paths3 = []
dif.diffuse_profiles.bragg_paths_M1 = []

for i in range( len( list(dif.diffuse_profiles.big_A1_pos_dict.keys()) ) ):
    key1 = list(dif.diffuse_profiles.big_A1_pos_dict.keys())[i]
    key2 = list(dif.diffuse_profiles.big_A2_pos_dict.keys())[i]
    key3 = list(dif.diffuse_profiles.big_A3_pos_dict.keys())[i]
    key4 = list(dif.diffuse_profiles.big_A4_pos_dict.keys())[i]

    print(key1)
    k_X2     = (dif.coef*(dif.diffuse_profiles.big_A2_pos_dict[key2] - dif.diffuse_profiles.big_A1_pos_dict[key1]) /2 )
    k_X3     = (dif.coef*(dif.diffuse_profiles.big_A3_pos_dict[key3] - dif.diffuse_profiles.big_A1_pos_dict[key1]) /2 )
    k_M1     = (dif.coef*(dif.diffuse_profiles.big_A4_pos_dict[key4] - dif.diffuse_profiles.big_A1_pos_dict[key1]) /2 ) # Flag of possible wrong coord    
    #now time dependent
    k_t_X2     = (dif.coef*(dif.diffuse_profiles.big_A2_pos_t_dict[key2] - dif.diffuse_profiles.big_A1_pos_t_dict[key1]) /2 )
    k_t_X3     = (dif.coef*(dif.diffuse_profiles.big_A3_pos_t_dict[key3] - dif.diffuse_profiles.big_A1_pos_t_dict[key1]) /2 )
    k_t_M1     = (dif.coef*(dif.diffuse_profiles.big_A4_pos_t_dict[key4] - dif.diffuse_profiles.big_A1_pos_t_dict[key1]) /2 ) #   
    
    
    q_tmp_X2 = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_2[key2] - dif.centerpos)    
    q_tmp_X3 = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_3[key3] - dif.centerpos) 
    q_tmp_M1 = dif.coef*(dif.diffuse_profiles.big_M_pos_dict[key4] - dif.centerpos) 
    # now time dependent
    
    q_tmp_t_X2 = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_time_2[key2] - dif.centerpos)    
    q_tmp_t_X3 = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_time_3[key3] - dif.centerpos) 
    q_tmp_t_M1 = dif.coef*(dif.diffuse_profiles.big_M_pos_dict_time[key4] - dif.centerpos) 
                         
    q_tmp_X2_20pc = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_2_20pc[key2] - dif.centerpos)    
    q_tmp_X3_20pc = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_3_20pc[key3] - dif.centerpos) 
    q_tmp_X2_60pc = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_2_60pc[key2] - dif.centerpos)    
    q_tmp_X3_60pc = dif.coef*(dif.diffuse_profiles.big_X_pos_dict_3_60pc[key3] - dif.centerpos)
    
    q_tmp_M1_20pc = dif.coef*(dif.diffuse_profiles.big_M_pos_dict_20pc[key4] - dif.centerpos) 
    q_tmp_M1_60pc = dif.coef*(dif.diffuse_profiles.big_M_pos_dict_60pc[key4] - dif.centerpos) 

    print(key1, key2,  np.dot(k_X2,q_tmp_X2) / (np.linalg.norm(q_tmp_X2)*np.linalg.norm(k_X2) ) )
    print(key1, key3,  np.dot(k_X3,q_tmp_X3) / (np.linalg.norm(q_tmp_X3)*np.linalg.norm(k_X3) ) )
    print(key1, key4,  np.dot(k_M1,q_tmp_M1) / (np.linalg.norm(q_tmp_M1)*np.linalg.norm(k_M1) ), dif.coef*(dif.diffuse_profiles.big_A4_pos_dict[key4]) )

#     print(key1, key2,  np.dot(k_X2,q_tmp_X2_20pc) / (np.linalg.norm(q_tmp_X2_20pc)*np.linalg.norm(k_X2) ) )
#     print(key1, key3,  np.dot(k_X3,q_tmp_X3_60pc) / (np.linalg.norm(q_tmp_X3_60pc)*np.linalg.norm(k_X3) ) )
    print('')
    
    dif.diffuse_profiles.L_projection_X2[i] = np.dot(k_X2,q_tmp_X2) / (np.linalg.norm(q_tmp_X2)*np.linalg.norm(k_X2) )
    dif.diffuse_profiles.L_projection_X3[i] = np.dot(k_X3,q_tmp_X3) / (np.linalg.norm(q_tmp_X3)*np.linalg.norm(k_X3) )
    dif.diffuse_profiles.L_projection_M1[i] = np.dot(k_M1,q_tmp_M1) / (np.linalg.norm(q_tmp_M1)*np.linalg.norm(k_M1) )
    
    dif.diffuse_profiles.L_projection_X2_20pc[i] = np.dot(k_X2,q_tmp_X2_20pc) / (np.linalg.norm(q_tmp_X2_20pc)*np.linalg.norm(k_X2) )
    dif.diffuse_profiles.L_projection_X3_20pc[i] = np.dot(k_X3,q_tmp_X3_20pc) / (np.linalg.norm(q_tmp_X3_20pc)*np.linalg.norm(k_X3) )
    dif.diffuse_profiles.L_projection_X2_60pc[i] = np.dot(k_X2,q_tmp_X2_60pc) / (np.linalg.norm(q_tmp_X2_60pc)*np.linalg.norm(k_X2) )
    dif.diffuse_profiles.L_projection_X3_60pc[i] = np.dot(k_X3,q_tmp_X3_60pc) / (np.linalg.norm(q_tmp_X3_60pc)*np.linalg.norm(k_X3) )
    dif.diffuse_profiles.L_projection_M1_20pc[i] = np.dot(k_M1,q_tmp_M1_20pc) / (np.linalg.norm(q_tmp_M1_20pc)*np.linalg.norm(k_M1) )
    dif.diffuse_profiles.L_projection_M1_60pc[i] = np.dot(k_M1,q_tmp_M1_60pc) / (np.linalg.norm(q_tmp_M1_60pc)*np.linalg.norm(k_M1) )
    
    dif.diffuse_profiles.q_X2[i] = q_tmp_X2
    dif.diffuse_profiles.q_X3[i] = q_tmp_X3
    dif.diffuse_profiles.q_M1[i] = q_tmp_M1
    dif.diffuse_profiles.k_X2[i] = k_X2
    dif.diffuse_profiles.k_X3[i] = k_X3
    dif.diffuse_profiles.k_M1[i] = k_M1
    
    #time dependent
    dif.diffuse_profiles.q_t_X2[i] = q_tmp_t_X2
    dif.diffuse_profiles.q_t_X3[i] = q_tmp_t_X3
    dif.diffuse_profiles.q_t_M1[i] = q_tmp_t_M1
    dif.diffuse_profiles.k_t_X2[i] = k_t_X2
    dif.diffuse_profiles.k_t_X3[i] = k_t_X3
    dif.diffuse_profiles.k_t_M1[i] = k_t_M1
    
    dif.diffuse_profiles.q_X2_20pc[i] = q_tmp_X2_20pc
    dif.diffuse_profiles.q_X3_20pc[i] = q_tmp_X3_20pc
    dif.diffuse_profiles.q_X2_60pc[i] = q_tmp_X2_60pc
    dif.diffuse_profiles.q_X3_60pc[i] = q_tmp_X3_60pc    
    dif.diffuse_profiles.q_M1_20pc[i] = q_tmp_M1_20pc
    dif.diffuse_profiles.q_M1_60pc[i] = q_tmp_M1_60pc
    
    dif.diffuse_profiles.I_X2_profile[i] = dif.diffuse_profiles.big_profile_dict_2[key1].mean(axis = (1))
    dif.diffuse_profiles.I_X3_profile[i] = dif.diffuse_profiles.big_profile_dict_3[key1].mean(axis = (1))
    dif.diffuse_profiles.I_M1_profile[i] = dif.diffuse_profiles.big_profile_dict_GM[key1].mean(axis = (1))
    
    dif.diffuse_profiles.I_X2[i] = dif.diffuse_profiles.big_profile_dict_2[key1][:,:,40:60].mean(axis = (1,2))
    dif.diffuse_profiles.I_X3[i] = dif.diffuse_profiles.big_profile_dict_3[key1][:,:,40:60].mean(axis = (1,2))
    dif.diffuse_profiles.I_M1[i] = dif.diffuse_profiles.big_profile_dict_GM[key1][:,:,40:60].mean(axis = (1,2))# Careful here too 
    
    dif.diffuse_profiles.I_X2_20pc[i] =  dif.diffuse_profiles.big_profile_dict_2[key1][:,:,20:40].mean(axis = (1,2))
    dif.diffuse_profiles.I_X3_20pc[i] =  dif.diffuse_profiles.big_profile_dict_3[key1][:,:,20:40].mean(axis = (1,2))
    dif.diffuse_profiles.I_X2_60pc[i] =  dif.diffuse_profiles.big_profile_dict_2[key1][:,:,60:80].mean(axis = (1,2))
    dif.diffuse_profiles.I_X3_60pc[i] =  dif.diffuse_profiles.big_profile_dict_3[key1][:,:,60:80].mean(axis = (1,2))    
    
    dif.diffuse_profiles.I_M1_20pc[i] =  dif.diffuse_profiles.big_profile_dict_GM[key1][:,:,20:40].mean(axis = (1,2))# Careful here too 
    dif.diffuse_profiles.I_M1_60pc[i] =  dif.diffuse_profiles.big_profile_dict_GM[key1][:,:,60:80].mean(axis = (1,2))   # Careful here too  
    
    dif.diffuse_profiles.bragg_paths2.append([key1, key2])
    dif.diffuse_profiles.bragg_paths3.append([key1, key3])
    dif.diffuse_profiles.bragg_paths_M1.append([key1, key4]) 
    
    

In [ ]:
plt.plot(dif.bincenters, np.linalg.norm(k_t_X2, axis =1) )
plt.xlim(-2,6)

In [ ]:
dif.diffuse_profiles.GX_path12 = np.append(np.asarray(list(dif.diffuse_profiles.big_A1_pos_dict.keys())), np.asarray(list(dif.diffuse_profiles.big_A2_pos_dict.keys())), axis = 1)
dif.diffuse_profiles.GX_path13 = np.append(np.asarray(list(dif.diffuse_profiles.big_A1_pos_dict.keys())), np.asarray(list(dif.diffuse_profiles.big_A3_pos_dict.keys())), axis = 1)
dif.diffuse_profiles.GX_paths = np.append(dif.diffuse_profiles.GX_path12,dif.diffuse_profiles.GX_path13, axis = 0)

dif.diffuse_profiles.GM_paths = np.append(np.asarray(list(dif.diffuse_profiles.big_A1_pos_dict.keys())), 
                                          np.asarray(list(dif.diffuse_profiles.big_A4_pos_dict.keys())), axis = 1)



In [ ]:
dif.diffuse_profiles.L_projection = np.append(dif.diffuse_profiles.L_projection_X2,  dif.diffuse_profiles.L_projection_X3)
dif.diffuse_profiles.L_projection_20pc = np.append(dif.diffuse_profiles.L_projection_X2_20pc,  dif.diffuse_profiles.L_projection_X3_20pc)
dif.diffuse_profiles.L_projection_60pc = np.append(dif.diffuse_profiles.L_projection_X2_60pc,  dif.diffuse_profiles.L_projection_X3_60pc)

# bragg_list_full = np.append


dif.diffuse_profiles.I_X_profile = np.append(dif.diffuse_profiles.I_X2_profile,dif.diffuse_profiles.I_X3_profile, axis = 0)
dif.diffuse_profiles.I_X = np.append(dif.diffuse_profiles.I_X2, dif.diffuse_profiles.I_X3, axis = 0)
dif.diffuse_profiles.I_X_20pc = np.append(dif.diffuse_profiles.I_X2_20pc, dif.diffuse_profiles.I_X3_20pc, axis = 0)
dif.diffuse_profiles.I_X_60pc = np.append(dif.diffuse_profiles.I_X2_60pc, dif.diffuse_profiles.I_X3_60pc, axis = 0)

dif.diffuse_profiles.q_X = np.append(dif.diffuse_profiles.q_X2,  dif.diffuse_profiles.q_X3, axis = 0)
dif.diffuse_profiles.k_X = np.append(dif.diffuse_profiles.k_X2,  dif.diffuse_profiles.k_X3, axis = 0)

dif.diffuse_profiles.q_t_X = np.append(dif.diffuse_profiles.q_t_X2,  dif.diffuse_profiles.q_t_X3, axis = 0)
dif.diffuse_profiles.k_t_X = np.append(dif.diffuse_profiles.k_t_X2,  dif.diffuse_profiles.k_t_X3, axis = 0)

dif.diffuse_profiles.q_X_20pc = np.append(dif.diffuse_profiles.q_X2_20pc,  dif.diffuse_profiles.q_X3_20pc, axis = 0)
dif.diffuse_profiles.q_X_60pc = np.append(dif.diffuse_profiles.q_X2_60pc,  dif.diffuse_profiles.q_X3_60pc, axis = 0)

dif.diffuse_profiles.Bragg_key_2 = (np.asarray(list(dif.diffuse_profiles.big_A1_pos_dict.keys())) + np.asarray(list(dif.diffuse_profiles.big_A2_pos_dict.keys())))/2
dif.diffuse_profiles.Bragg_key_3 = (np.asarray(list(dif.diffuse_profiles.big_A1_pos_dict.keys())) + np.asarray(list(dif.diffuse_profiles.big_A3_pos_dict.keys())))/2
dif.diffuse_profiles.Bragg_key_4 = (np.asarray(list(dif.diffuse_profiles.big_A1_pos_dict.keys())) + np.asarray(list(dif.diffuse_profiles.big_A4_pos_dict.keys())))/2

dif.diffuse_profiles.Bragg_key_2_20pc = np.floor(dif.diffuse_profiles.Bragg_key_2) + (dif.diffuse_profiles.Bragg_key_2 %1)*(0.3/0.5)
dif.diffuse_profiles.Bragg_key_3_20pc = np.floor(dif.diffuse_profiles.Bragg_key_3) + (dif.diffuse_profiles.Bragg_key_3 %1)*(0.3/0.5)
dif.diffuse_profiles.Bragg_key_4_20pc = np.floor(dif.diffuse_profiles.Bragg_key_4) + (dif.diffuse_profiles.Bragg_key_4 %1)*(0.3/0.5)

dif.diffuse_profiles.Bragg_key_2_60pc = np.floor(dif.diffuse_profiles.Bragg_key_2) + (dif.diffuse_profiles.Bragg_key_2 %1)*(0.6/0.5)
dif.diffuse_profiles.Bragg_key_3_60pc = np.floor(dif.diffuse_profiles.Bragg_key_3) + (dif.diffuse_profiles.Bragg_key_3 %1)*(0.6/0.5)
dif.diffuse_profiles.Bragg_key_4_60pc = np.floor(dif.diffuse_profiles.Bragg_key_4) + (dif.diffuse_profiles.Bragg_key_4 %1)*(0.6/0.5)


dif.diffuse_profiles.Bragg_key = np.append(dif.diffuse_profiles.Bragg_key_2, dif.diffuse_profiles.Bragg_key_3, axis = 0)
dif.diffuse_profiles.Bragg_key_20pc = np.append(dif.diffuse_profiles.Bragg_key_2_20pc, dif.diffuse_profiles.Bragg_key_3_20pc, axis = 0)
dif.diffuse_profiles.Bragg_key_60pc = np.append(dif.diffuse_profiles.Bragg_key_2_60pc, dif.diffuse_profiles.Bragg_key_3_60pc, axis = 0)

dif.diffuse_profiles.Bragg_key_M = dif.diffuse_profiles.Bragg_key_4
dif.diffuse_profiles.Bragg_key_M_20pc = dif.diffuse_profiles.Bragg_key_4_20pc
dif.diffuse_profiles.Bragg_key_M_60pc = dif.diffuse_profiles.Bragg_key_4_60pc


In [ ]:
idx_list = np.argsort(dif.diffuse_profiles.L_projection)
idx_list_M = np.argsort(dif.diffuse_profiles.L_projection_M1)


#X point
dif.diffuse_profiles.L_projection_sorted = dif.diffuse_profiles.L_projection[idx_list]
dif.diffuse_profiles.L_projection_20pc_sorted = dif.diffuse_profiles.L_projection_20pc[idx_list]
dif.diffuse_profiles.L_projection_60pc_sorted = dif.diffuse_profiles.L_projection_60pc[idx_list]
# M point
dif.diffuse_profiles.L_projection_M_sorted = dif.diffuse_profiles.L_projection_M1[idx_list_M]
dif.diffuse_profiles.L_projection_M_20pc_sorted = dif.diffuse_profiles.L_projection_M1_20pc[idx_list_M]
dif.diffuse_profiles.L_projection_M_60pc_sorted = dif.diffuse_profiles.L_projection_M1_60pc[idx_list_M]


#Xpoint
dif.diffuse_profiles.I_X_profile_sorted = dif.diffuse_profiles.I_X_profile[idx_list]
dif.diffuse_profiles.I_X_sorted = dif.diffuse_profiles.I_X[idx_list]
dif.diffuse_profiles.I_X_20pc_sorted = dif.diffuse_profiles.I_X_20pc[idx_list]
dif.diffuse_profiles.I_X_60pc_sorted = dif.diffuse_profiles.I_X_60pc[idx_list]
#M point
dif.diffuse_profiles.I_M_profile_sorted = dif.diffuse_profiles.I_M1_profile[idx_list_M]
dif.diffuse_profiles.I_M_sorted = dif.diffuse_profiles.I_M1[idx_list_M]
dif.diffuse_profiles.I_M_20pc_sorted = dif.diffuse_profiles.I_M1_20pc[idx_list_M]
dif.diffuse_profiles.I_M_60pc_sorted = dif.diffuse_profiles.I_M1_60pc[idx_list_M]


#X point
dif.diffuse_profiles.q_X_sorted = dif.diffuse_profiles.q_X[idx_list]
dif.diffuse_profiles.k_X_sorted = dif.diffuse_profiles.k_X[idx_list]
# Xpoint, but time resolved
dif.diffuse_profiles.q_t_X_sorted = dif.diffuse_profiles.q_t_X[idx_list]
dif.diffuse_profiles.k_t_X_sorted = dif.diffuse_profiles.k_t_X[idx_list]

dif.diffuse_profiles.q_X_20pc_sorted = dif.diffuse_profiles.q_X_20pc[idx_list]
dif.diffuse_profiles.q_X_60pc_sorted = dif.diffuse_profiles.q_X_60pc[idx_list]
#M point
dif.diffuse_profiles.q_M_sorted = dif.diffuse_profiles.q_M1[idx_list_M]
dif.diffuse_profiles.q_t_M_sorted = dif.diffuse_profiles.q_t_M1[idx_list_M]

dif.diffuse_profiles.q_M_20pc_sorted = dif.diffuse_profiles.q_M1_20pc[idx_list_M]
dif.diffuse_profiles.q_M_60pc_sorted = dif.diffuse_profiles.q_M1_60pc[idx_list_M]


#X point
dif.diffuse_profiles.Bragg_key_sorted = dif.diffuse_profiles.Bragg_key[idx_list]
dif.diffuse_profiles.Bragg_key_sorted_20pc = dif.diffuse_profiles.Bragg_key_20pc[idx_list]
dif.diffuse_profiles.Bragg_key_sorted_60pc = dif.diffuse_profiles.Bragg_key_60pc[idx_list]
#M point
dif.diffuse_profiles.Bragg_key_M_sorted = dif.diffuse_profiles.Bragg_key_M[idx_list_M]
dif.diffuse_profiles.Bragg_key_M_sorted_20pc = dif.diffuse_profiles.Bragg_key_M_20pc[idx_list_M]
dif.diffuse_profiles.Bragg_key_M_sorted_60pc = dif.diffuse_profiles.Bragg_key_M_60pc[idx_list_M]

dif.diffuse_profiles.GX_paths_sorted = dif.diffuse_profiles.GX_paths[idx_list]
dif.diffuse_profiles.GM_paths_sorted = dif.diffuse_profiles.GM_paths[idx_list_M]



In [ ]:
sel_time = (dif.bincenters >-2.)&(dif.bincenters < 9.)
dif.diffuse_profiles.time_fits_raw_popt = np.zeros((dif.diffuse_profiles.I_X.shape[0], 4))
dif.diffuse_profiles.time_fits_raw_pcov = np.zeros((dif.diffuse_profiles.I_X.shape[0], 4, 4))

dif.diffuse_profiles.time_fits_raw_M_popt = np.zeros((dif.diffuse_profiles.I_M1.shape[0], 4))
dif.diffuse_profiles.time_fits_raw_M_pcov = np.zeros((dif.diffuse_profiles.I_M1.shape[0], 4, 4))

dif.diffuse_profiles.time_fits_raw__with_recov_popt = np.zeros((dif.diffuse_profiles.I_X.shape[0], 6))
dif.diffuse_profiles.time_fits_raw__with_recov_pcov = np.zeros((dif.diffuse_profiles.I_X.shape[0], 6, 6))
dif.diffuse_profiles.time_fits_raw__with_recov_lmfit = dict()

dif.diffuse_profiles.time_fits_raw_M_with_recov_popt = np.zeros((dif.diffuse_profiles.I_M1.shape[0], 6))
dif.diffuse_profiles.time_fits_raw_M_with_recov_pcov = np.zeros((dif.diffuse_profiles.I_M1.shape[0], 6, 6))
dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit = dict()

In [ ]:
# fig, ax = plt.subplots(figsize = (8,8), dpi = 150)
fig, ax = plt.subplots(figsize = (8,8))

ax.imshow(image - dif.bkg_img_bkg, vmin = -0, vmax = 10,cmap = 'binary',)
cmap = plt.get_cmap('cool')
for i in np.arange(len(list(dif.diffuse_profiles.big_X_pos_dict_3.keys()))):
    key2_tmp = list(dif.diffuse_profiles.big_X_pos_dict_2.keys())[i]
    key3_tmp = list(dif.diffuse_profiles.big_X_pos_dict_3.keys())[i]
    key4_tmp = list(dif.diffuse_profiles.big_M_pos_dict.keys())[i]

    
    color = cmap(dif.diffuse_profiles.L_projection_X2[i])
    plt.scatter(dif.diffuse_profiles.big_X_pos_dict_2[key2_tmp][0], dif.diffuse_profiles.big_X_pos_dict_2[key2_tmp][1], marker='o', color=color)
    
    color = cmap(dif.diffuse_profiles.L_projection_X3[i])
    plt.scatter(dif.diffuse_profiles.big_X_pos_dict_3[key3_tmp][0], dif.diffuse_profiles.big_X_pos_dict_3[key3_tmp][1], marker='o', color=color)
    
    color = cmap(dif.diffuse_profiles.L_projection_M1[i])
    plt.scatter(dif.diffuse_profiles.big_M_pos_dict[key4_tmp][0], dif.diffuse_profiles.big_M_pos_dict[key4_tmp][1], marker='s', color=color)
    
    
    plt.scatter(dif.diffuse_profiles.big_X_pos_dict_3_20pc[key3_tmp][0], dif.diffuse_profiles.big_X_pos_dict_3_20pc[key3_tmp][1], marker='.', color='k')
    plt.scatter(dif.diffuse_profiles.big_X_pos_dict_2_20pc[key2_tmp][0], dif.diffuse_profiles.big_X_pos_dict_2_20pc[key2_tmp][1], marker='.', color='k')
    
    plt.scatter(dif.diffuse_profiles.big_X_pos_dict_3_60pc[key3_tmp][0], dif.diffuse_profiles.big_X_pos_dict_3_60pc[key3_tmp][1], marker='x', color='k')
    plt.scatter(dif.diffuse_profiles.big_X_pos_dict_2_60pc[key2_tmp][0], dif.diffuse_profiles.big_X_pos_dict_2_60pc[key2_tmp][1], marker='x', color='k')


In [ ]:
model_exp_sat = Model(dt.two_expssum_2_with_rec_tau)
# model_exp_sat.param_names

            
#             result_tmp_1 = dif.dmodel_bragg.fit(roi_1.ravel(), params_bragg_t0_1, xx_yy=xx_yy_1)
# #             print(result_tmp.best_values)
#             pcov_1 = result_tmp_1.covar

In [ ]:
def exp_sat_tau_rec(time, A1,T1, A2,T2, A_rec,T_rec ):
    '''
    single exponential saturation:
    
    Parameters: 
    time: array with time delays
    A: Amplitude 
    B: 1/Tau: time constant
    
    Retruns:
    y : intensities
    '''
    t0= 0 
#     y = A-A*np.exp(-(time-t0)/T) 
    y = A1*(1-np.exp(-(time-t0)/T1))*np.exp(-(time-t0)/T_rec)
    mask = np.heaviside(time-t0, 0 )
    y *= mask

    return y

In [ ]:
model_exp_sat = Model(exp_sat_tau_rec)


In [ ]:
for i in np.arange(dif.diffuse_profiles.I_X.shape[0] -0 ):
    
    color = cmap(dif.diffuse_profiles.L_projection_sorted[i])
    
    time_tmp = dif.bincenters#[sel_time]
    val_tmp = (dif.diffuse_profiles.I_X_sorted[i] - dif.diffuse_profiles.I_X_sorted[i][0:5].mean())#[sel_time]*1
    sel_time = np.where(time_tmp < 106 )
    
    p0 = [val_tmp[-1]/2, 1, 0, 1 ]
    
    print(
    np.round(p0[0], 3), np.round(1/p0[1], 3), 
    np.round(p0[2], 3), np.round(1/p0[3], 3), )
    
    popt, pcov = opt.curve_fit(dt.two_expssum_free , time_tmp, val_tmp, p0=p0, bounds =[[0,0,0,0],[np.inf,np.inf,np.inf,np.inf ]] )
    dif.diffuse_profiles.time_fits_raw_popt[i] = popt
    print(
    np.round(popt[0], 3), np.round(1/popt[1], 3), 
    np.round(popt[2], 3), np.round(1/popt[3], 3), )
    
    time_tmp = dif.bincenters[sel_time]
    val_tmp = 1*(dif.diffuse_profiles.I_X_sorted[i] - dif.diffuse_profiles.I_X_sorted[i][0:5].mean())[sel_time]
    p0 = np.append(popt, np.asarray([val_tmp[-1], 30 ]) )
    
        
    params_exp_sat = Parameters()
    # (amp0_1, x0_1, y0_1, sigmax_1, sigmay_1,theta0_1, offset0_1, initial_ax_1, initial_by_1)
#     params_exp_sat.add('A1', value=popt[0], ) #0.002,
#     params_exp_sat.add('T1', value=popt[1], ) #0.5,
    params_exp_sat.add('A1', value=0.002, ) #0.002,
    params_exp_sat.add('T1', value=0.5, ) #0.5,
#     params_exp_sat.add('A2', value=popt[2])
#     params_exp_sat.add('T2', value=popt[3], )
    
    params_exp_sat.add('A2', value=0, vary=False)
    params_exp_sat.add('T2', value=1, vary=False)

#     params_exp_sat.add('A_rec', value= np.asarray(val_tmp[-1]), )
#     params_exp_sat.add('A_rec', value= 0.000, vary = False )#0.0005
    params_exp_sat.add('A_rec', expr='A1' )#0.0005

    params_exp_sat.add('T_rec', value=50, vary=True ) 
# dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit    
    try:
        exp_sat_res_tmp = model_exp_sat.fit(val_tmp, params_exp_sat, time=time_tmp)
        dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i] = copy.copy(exp_sat_res_tmp)

        v_tmp = np.asarray([
            dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.params['A1'].value,
            dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.params['T1'].value,
            dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.params['A2'].value,
            dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.params['T2'].value,
            dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.params['A_rec'].value,
            dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.params['T_rec'].value
        ])
        dif.diffuse_profiles.time_fits_raw__with_recov_popt[i] = v_tmp #popt
        dif.diffuse_profiles.time_fits_raw__with_recov_pcov[i] = np.sqrt(np.diag(dif.diffuse_profiles.time_fits_raw__with_recov_lmfit[i].result.covar))

        print(
        np.round(p0[0], 3), np.round(p0[1], 3), 
        np.round(p0[2], 3), np.round(p0[3], 3), 
        np.round(p0[4], 3), np.round(p0[5], 3),  
        )

        print(
            np.round(dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][0], 3), np.round(1/dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][1], 3), 
            np.round(dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][2], 3), np.round(1/dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][3], 3), 
            np.round(dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][4], 3), np.round(1/dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][5], 3),  
        )
        print('')



    except ValueError:
        pass
    
    
#     popt, pcov = opt.curve_fit(dt.two_expssum_2_with_rec_tau , time_tmp, val_tmp, p0=p0, bounds = [0, np.inf])#bounds =[[0,0,0, 0,-np.inf,0],[np.inf,np.inf,np.inf, np.inf,0.000001,np.inf ]])
#     dif.diffuse_profiles.time_fits_raw__with_recov_popt[i] = popt
#     dif.diffuse_profiles.time_fits_raw__with_recov_pcov[i] = pcov

In [ ]:
print(exp_sat_res_tmp.fit_report())

In [ ]:
for i in np.arange(dif.diffuse_profiles.I_M1.shape[0] -0 ):
    
    color = cmap(dif.diffuse_profiles.L_projection_M_sorted[i])
    
    time_tmp = dif.bincenters[sel_time]
    val_tmp = (dif.diffuse_profiles.I_M_sorted[i] - dif.diffuse_profiles.I_M_sorted[i][0])[sel_time]*1
    if ~np.isnan( dif.diffuse_profiles.I_M_sorted[i] ).any():
#     print(val_tmp[0])

        p0 = [val_tmp[-1]/2, 1, 0, 1 ]

        print(
        np.round(p0[0], 3), np.round(1/p0[1], 3), 
        np.round(p0[2], 3), np.round(1/p0[3], 3), )

        popt, pcov = opt.curve_fit(dt.two_expssum_free , time_tmp, val_tmp, p0=p0, bounds =[[0,0,0,0],[np.inf,np.inf,np.inf,np.inf ]])
        dif.diffuse_profiles.time_fits_raw_popt[i] = popt
        print(
        np.round(popt[0], 3), np.round(1/popt[1], 3), 
        np.round(popt[2], 3), np.round(1/popt[3], 3), )

        time_tmp = dif.bincenters#[sel_time]
        val_tmp = 1*(dif.diffuse_profiles.I_M_sorted[i] - dif.diffuse_profiles.I_M_sorted[i][0:5].mean())#[sel_time]
        
        sel_time = np.where(time_tmp < 106 )
        time_tmp = time_tmp[sel_time]
        val_tmp = val_tmp[sel_time]
        
        p0 = np.append(popt, np.asarray([val_tmp[-1], 30 ]) )
#         popt, pcov = opt.curve_fit(dt.two_expssum_2_with_rec_tau , time_tmp, val_tmp, p0=p0, bounds = [0, np.inf])#bounds =[[0,0,0, 0,-np.inf,0],[np.inf,np.inf,np.inf, np.inf,0.000001,np.inf ]])
#         dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i] = popt
#         dif.diffuse_profiles.time_fits_raw_M_with_recov_pcov[i] = pcov





        params_exp_sat = Parameters()
        # (amp0_1, x0_1, y0_1, sigmax_1, sigmay_1,theta0_1, offset0_1, initial_ax_1, initial_by_1)
        params_exp_sat.add('A1', value=0.002,)
        params_exp_sat.add('T1', value=0.5,)
        params_exp_sat.add('A2', value=0,vary=False)
        params_exp_sat.add('T2', value=1,vary=False)
        
#         params_exp_sat.add('A2', value=popt[2],)
#         params_exp_sat.add('T2', value=popt[3],)
#         params_exp_sat.add('A_rec', value= 0.00005, vary=True)# value= 0.00005, vary=True)#0.0005)
        params_exp_sat.add('A_rec', expr='A1',)# value= 0.00005, vary=True)#0.0005)
        params_exp_sat.add('T_rec', value=50, vary=True )

    # dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit    
        try:
            exp_sat_res_tmp = model_exp_sat.fit(val_tmp, params_exp_sat, time=time_tmp)
            dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i] = copy.copy(exp_sat_res_tmp)

            v_tmp = np.asarray([
                dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.params['A1'].value,
                dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.params['T1'].value,
                dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.params['A2'].value,
                dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.params['T2'].value,
                dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.params['A_rec'].value,
                dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.params['T_rec'].value
            ])
            dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i] = v_tmp #popt
            dif.diffuse_profiles.time_fits_raw_M_with_recov_pcov[i] = np.sqrt(np.diag(dif.diffuse_profiles.time_fits_raw_M_with_recov_lmfit[i].result.covar))

            print(
            np.round(p0[0], 3), np.round(p0[1], 3), 
            np.round(p0[2], 3), np.round(p0[3], 3), 
            np.round(p0[4], 3), np.round(p0[5], 3),  
            )

            print(
                np.round(dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][0], 3), np.round(1/dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][1], 3), 
                np.round(dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][2], 3), np.round(1/dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][3], 3), 
                np.round(dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][4], 3), np.round(1/dif.diffuse_profiles.time_fits_raw__with_recov_popt[i][5], 3),  
            )
            print('')



        except ValueError:
            pass







#         print(
#         np.round(p0[0], 3), np.round(1/p0[1], 3), 
#         np.round(p0[2], 3), np.round(1/p0[3], 3), 
#         np.round(p0[4], 3), np.round(1/p0[5], 3),  
#         )

#         print(
#             np.round(dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i][0], 3), np.round(1/dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i][1], 3), 
#             np.round(dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i][2], 3), np.round(1/dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i][3], 3), 
#             np.round(dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i][4], 3), np.round(1/dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i][5], 3),  
#         )
#         print('')
    else:
        dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i] = dif.diffuse_profiles.time_fits_raw_M_with_recov_popt[i]*np.nan
        dif.diffuse_profiles.time_fits_raw_M_with_recov_pcov[i] = dif.diffuse_profiles.time_fits_raw_M_with_recov_pcov[i]*np.nan


In [ ]:
plt.figure(figsize = (12,8))
fig, [ax1, ax2] = plt.subplots(ncols=2, figsize = (16,8))
fig.subplots_adjust(wspace=.05)

for i in np.arange(dif.diffuse_profiles.I_X.shape[0] -0 ):
    
    color = cmap(dif.diffuse_profiles.L_projection_sorted[i])
 
    ax1.plot(dif.bincenters,1*(dif.diffuse_profiles.I_X_sorted[i] - dif.diffuse_profiles.I_X_sorted[i][0]),'.', color = color, alpha = 0.8 , label = f'${dif.diffuse_profiles.Bragg_key_sorted[i][0], dif.diffuse_profiles.Bragg_key_sorted[i][1]}: Q \cdot k = $ {np.round(dif.diffuse_profiles.L_projection_sorted[i], 3)}')
    ax1.plot(dif.bincenters,model_exp_sat.func(dif.bincenters, *dif.diffuse_profiles.time_fits_raw__with_recov_popt[i]),'--', color = color )

    ax2.plot(dif.bincenters,1*(dif.diffuse_profiles.I_X_sorted[i] - dif.diffuse_profiles.I_X_sorted[i][0]),'.', color = color, alpha = 0.8 , label = f'${dif.diffuse_profiles.Bragg_key_sorted[i][0], dif.diffuse_profiles.Bragg_key_sorted[i][1]}: Q \cdot k = $ {np.round(dif.diffuse_profiles.L_projection_sorted[i], 3)}')
    ax2.plot(dif.bincenters,model_exp_sat.func(dif.bincenters, *dif.diffuse_profiles.time_fits_raw__with_recov_popt[i]),'--', color = color )


    
# ax1.legend(ncol = 2, fontsize = 11)
ax1.set_xlabel('Delay (ps)', fontsize = 18)
ax1.set_ylabel(r'$I - I_0$', fontsize = 18)

ax2.set_xlabel('Delay (ps)', fontsize = 18)
# ax2.set_ylabel(r'$I - I_0$', fontsize = 18)
ax2.set_xlim(-2,6)


ax1.tick_params(axis='both', labelsize=18)
ax2.tick_params(axis='both', labelsize=18)

# ax1.set_xticks([]);

# ax2.set_xticks([]);
ax2.set_yticks([]);


# plt.tight_layout()


In [ ]:
plt.figure()
plt.plot( np.linalg.norm(dif.diffuse_profiles.q_X_sorted, axis = 1), dif.diffuse_profiles.I_X_sorted[:,0], '.')

In [ ]:
t_idx = 0
dif.diffuse_profiles.k_t_X_sorted.shape
dif.diffuse_profiles.q_t_X_sorted[:,t_idx,:].shape

In [72]:
# import pickle 
import dill as pickle
# here is how I save it
with open('./data_diffuse/dif_big_dataset_diffuse.pkl', 'wb') as f:
    pickle.dump(dif, f)

# And here is how I would load it
# with open('saved_dictionary.pkl', 'rb') as f:
#     loaded_dict = pickle.load(f)